In [1]:
import itertools
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Input, Dropout

In [2]:
from google.colab import files
uploaded = files.upload()


Saving df.csv to df.csv


In [3]:
import io
dframe = pd.read_csv(io.BytesIO(uploaded['df.csv']))

In [4]:
#dframe = pd.read_csv("df.csv", encoding = "ISO-8859-1")
x_df = dframe.drop(['satisfaction'], axis=1) #Se eliminan time y amount porque no son un parametro util como los V...., Class se elimina porque estamos entrenando al modelo, y para eso no le tenemos que entregar las respuestas directamente
vectorizer = DictVectorizer(sparse=False)
X = vectorizer.fit_transform(x_df.to_dict("records"))
X.shape


(124855, 23)

In [5]:
y = dframe.satisfaction.values
classes = np.unique(y)
y.shape

(124855,)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=109, stratify=y)
X_train, X_val , y_train, y_val =train_test_split(X_train, y_train, test_size=0.2,random_state=109)

In [ ]:
X_train.shape

(79907, 23)

In [ ]:
#embeddings_layer = Embedding(input_dim=inp_dim, output_dim=8, input_length=11, trainable=True)

dense1 = Dense(36, activation="sigmoid")
dense2 = Dense(256, activation="tanh")#####se intento poner capa 2 aumentarla a 60 y la 3 de 160 a 210
dense3 = Dense(512, activation="sigmoid")
dense5 = Dense(64, activation="relu")
dense6 = Dense(9, activation="sigmoid")

In [ ]:
input = Input(shape=(23, ))
print(input)
x = dense1(input)
x = dense2(x)
x = dense3(x)
x = dense5(x)
outputs = dense6(x)


KerasTensor(type_spec=TensorSpec(shape=(None, 23), dtype=tf.float32, name='input_3'), name='input_3', description="created by layer 'input_3'")


In [ ]:
model = Model(inputs=input, outputs=outputs)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 23)]              0         
                                                                 
 dense_5 (Dense)             (None, 36)                864       
                                                                 
 dense_6 (Dense)             (None, 256)               9472      
                                                                 
 dense_7 (Dense)             (None, 512)               131584    
                                                                 
 dense_8 (Dense)             (None, 64)                32832     
                                                                 
 dense_9 (Dense)             (None, 9)                 585       
                                                                 
Total params: 175,337
Trainable params: 175,337
Non-trainab

In [ ]:
model.compile(optimizer="adadelta", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) # se cambio de adam a adadelta y mejoro a 61. si se parte como adadelta  no da buenos resultados, pero si se parte con adam y luego pasamos a adadleta mejore

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10) # se cambio monitor de los a val accuracy, a medida va mejorando accuracy ir disminiuyendo patience

In [ ]:
model.fit(X_train, y_train, batch_size=20, epochs=50, callbacks=[callback],validation_data=(X_val, y_val))  #se intento con batch size de 30 y con epochs de 20

Epoch 1/50
3996/3996 [==============================] - 19s 4ms/step - loss: 1.1740 - accuracy: 0.5425 - val_loss: 0.7629 - val_accuracy: 0.5679
Epoch 2/50
3996/3996 [==============================] - 18s 4ms/step - loss: 0.7150 - accuracy: 0.5642 - val_loss: 0.6884 - val_accuracy: 0.5679
Epoch 3/50
3996/3996 [==============================] - 18s 5ms/step - loss: 0.6793 - accuracy: 0.5642 - val_loss: 0.6704 - val_accuracy: 0.5679
Epoch 4/50
3996/3996 [==============================] - 17s 4ms/step - loss: 0.6651 - accuracy: 0.5642 - val_loss: 0.6585 - val_accuracy: 0.5679
Epoch 5/50
3996/3996 [==============================] - 18s 4ms/step - loss: 0.6535 - accuracy: 0.5642 - val_loss: 0.6469 - val_accuracy: 0.5679
Epoch 6/50
3996/3996 [==============================] - 17s 4ms/step - loss: 0.6414 - accuracy: 0.5670 - val_loss: 0.6344 - val_accuracy: 0.6145
Epoch 7/50
3996/3996 [==============================] - 18s 4ms/step - loss: 0.6278 - accuracy: 0.6312 - val_loss: 0.6198 - val_ac

In [ ]:
labels = [0,1]
Y_preds = model.predict(X_test).argmax(axis=-1)

print("Test Accuracy : {}".format(accuracy_score(y_test, Y_preds)))
print("\nClassification Report : ")
print(classification_report(y_test, Y_preds))

781/781 [==============================] - 1s 2ms/step
Test Accuracy : 0.8658844259340835

Classification Report : 
              precision    recall  f1-score   support

         0.0       0.87      0.90      0.88     14106
         1.0       0.86      0.82      0.84     10865

    accuracy                           0.87     24971
   macro avg       0.87      0.86      0.86     24971
weighted avg       0.87      0.87      0.87     24971



In [7]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()

clf.fit(X_train, y_train)

preds = clf.predict(X_test)

acc = accuracy_score(y_test, preds) * 100


print('Out-of-the-box MLP classifier')
print('Accuracy: {:.2f} %'.format(acc))

Out-of-the-box MLP classifier
Accuracy: 95.92 %


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [9]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.96      0.97      0.96     14106
         1.0       0.96      0.95      0.95     10865

    accuracy                           0.96     24971
   macro avg       0.96      0.96      0.96     24971
weighted avg       0.96      0.96      0.96     24971

